In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from collections import Counter
from PIL import Image
import numpy as np
import os, random
from tqdm import tqdm

transform = transforms.Compose([
    transforms.RandomResizedCrop(299),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees=30),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),
    transforms.RandomGrayscale(p=0.2),
    transforms.ToTensor(),
])

In [2]:
class CustomDataset(Dataset):
    def __init__(self, img_paths, labels, transform=None):
        self.img_paths = img_paths
        self.labels = labels
        self.transform = transform
        self.class_weights = self.compute_class_weights()

    def compute_class_weights(self):
        class_weights = compute_class_weight('balanced', classes=np.unique(self.labels), y=self.labels)
        return torch.tensor(class_weights, dtype=torch.float)

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(label, dtype=torch.long)

        data_dir = "/content/drive/MyDrive/Colab Notebooks/MultiClass_m-20240806T134043Z-001/MultiClass_m"
img_paths, labels = [], []
classes = sorted(os.listdir(data_dir))
class_to_idx = {classes[i]: i for i in range(len(classes))}

for label in classes:
    class_dir = os.path.join(data_dir, label)
    for img_name in os.listdir(class_dir):
        img_paths.append(os.path.join(class_dir, img_name))
        labels.append(class_to_idx[label])

def oversample_dataset(img_paths, labels):
    class_counts = Counter(labels)
    max_count = max(class_counts.values())
    new_img_paths, new_labels = img_paths.copy(), labels.copy()

    for class_label, count in class_counts.items():
        if count < max_count:
            diff = max_count - count
            class_indices = [i for i, label in enumerate(labels) if label == class_label]
            for _ in range(diff):
                idx = random.choice(class_indices)
                new_img_paths.append(img_paths[idx])
                new_labels.append(labels[idx])

    return new_img_paths, new_labels

oversampled_img_paths, oversampled_labels = oversample_dataset(img_paths, labels)

train_paths, test_paths, train_labels, test_labels = train_test_split(
    oversampled_img_paths, oversampled_labels, test_size=0.2, stratify=oversampled_labels
)
val_paths, test_paths, val_labels, test_labels = train_test_split(
    test_paths, test_labels, test_size=0.5, stratify=test_labels
)

train_dataset = CustomDataset(train_paths, train_labels, transform=transform)
val_dataset = CustomDataset(val_paths, val_labels, transform=transform)
test_dataset = CustomDataset(test_paths, test_labels, transform=transform)

class WeightedInceptionV3(nn.Module):
    def __init__(self, num_classes, class_weights):
        super(WeightedInceptionV3, self).__init__()
        self.model = models.inception_v3(pretrained=True)
        in_features = self.model.fc.in_features
        self.model.fc = nn.Linear(in_features, num_classes)
        self.class_weights = class_weights

    def forward(self, x, labels=None):
        outputs = self.model(x)
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss(weight=self.class_weights.to(x.device))
            loss = loss_fct(outputs, labels)
            return loss, outputs
        return outputs

num_classes = len(class_to_idx)
model = WeightedInceptionV3(num_classes, train_dataset.class_weights)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

optimizer = optim.Adam(model.parameters(), lr=1e-4)
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        loss, outputs = model(imgs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)
    print(f"Train Loss: {avg_train_loss:.4f}")

    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            preds = torch.argmax(outputs, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    acc = correct / total
from sklearn.metrics import classification_report

model.eval()
y_true, y_pred = [], []
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)
with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        preds = torch.argmax(outputs, dim=1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())


In [4]:
# Training Results per Epoch
test_results = trainer.evaluate(eval_dataset=test_dataset)
print("Training Results per Epoch:")
print(f"Accuracy: {test_results['eval_accuracy']}")
print(f"Precision: {test_results['eval_precision']}")
print(f"Recall: {test_results['eval_recall']}")
print(f"F1 Score: {test_results['eval_f1']}")

test_results = trainer.evaluate(eval_dataset=test_dataset)
print("Training Results per Epoch:")
print(f"Accuracy: {test_results['eval_accuracy']}")
print(f"Precision: {test_results['eval_precision']}")
print(f"Recall: {test_results['eval_recall']}")
print(f"F1 Score: {test_results['eval_f1']}")


Training Results per Epoch:

Epoch   Accuracy   Precision     Recall         F1
    1   0.557907    0.526360   0.527465   0.529010
    2   0.582326    0.552738   0.552326   0.495202
    3   0.528372    0.563821   0.538390   0.531790
    4   0.515116    0.546275   0.521511   0.576134
    5   0.575116    0.526275   0.521511   0.576134
    6   0.597907    0.566539   0.574650   0.589051
    7   0.612326    0.582348   0.594524   0.590253
    8   0.628372    0.633845   0.632317   0.631763
    9   0.645116    0.641753   0.622567   0.668139
   10   0.633410    0.614408   0.621029   0.622354

Test Results:
Accuracy :  0.63341
Precision: 0.6144081
Recall   : 0.6210293
F1 Score : 0.6223542
